In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [30]:
def get_page(link = '/soccer'):
    page_link = 'http://www.betexplorer.com'+link    
    try:
        # fetch the content from url
        page_response = requests.get(page_link, timeout=5)
        # parse html
        page_content = BeautifulSoup(page_response.content, "html.parser")
    except:
        #caso a página não responda, espera por 5min e tenta novamente
        time.sleep(300)
        page_response = requests.get(page_link, timeout=60)
        # parse html
        page_content = BeautifulSoup(page_response.content, "html.parser")
    return page_content


def scrapGroupPhase(page, page_link, tournament, season):
    #df_goals = pd.DataFrame(columns=['Match_Id','Team_Id', 'Time','Player_Name']) #para coleta d
    df = pd.DataFrame(columns=['Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
    #div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
    th_position = 0
    td_position = 0
    data = []
    for content in page:
        for li in content.find_all('li'):            
            fase = li.text
            link = li.select('a')[0].attrs['href']
            page_content = get_page(page_link+link)
            table = page_content.find_all('table')[0]
            for row in table.find_all(['td', 'th']):
                if row.name == 'th' and th_position == 0:
                    rodada = row.text
                    data = [tournament, season, rodada]
                    th_position = 1
                elif row.name == 'td':
                    if th_position == 0:
                        rodada = fase
                        data = [tournament, season, rodada]
                        th_position = 1
                    if td_position == 0:
                        data.append(row.text.split(' - ')[0])
                        data.append(row.text.split(' - ')[1])
                        for a in row.find_all('a', href=True):
                            link = a['href']
                            #df_goals = scrapGoals(len(df),[row.text.split(' - ')[0],row.text.split(' - ')[1]],link, df_goals)
                    elif 2 <= td_position <= 4:
                        try:
                            data.append(row.select('span')[2].attrs['data-odd'])
                        except:
                            try: 
                                data.append(row.attrs['data-odd'])
                            except:
                                data.append(row.text)                    
                    else:
                        data.append(row.text)
                    td_position+=1       
                    if td_position == 6:
                        data.append(link)
                        data.append(fase)
                        df.loc[len(df)] = data
                        data = [tournament, season, rodada]
                        th_position = 0
                        td_position = 0
    #return df, df_goals
    return df

In [31]:
def scrapChampionship(page_content, tournament, season):
    #df_goals = pd.DataFrame(columns=['Match_Id','Team_Id', 'Time','Player_Name'])
    df = pd.DataFrame(columns=['Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
    th_position = 0
    td_position = 0
    data = []
    table = page_content.find_all('table')[0]
    for row in table.find_all(['td', 'th']):
        if row.name == 'th' and th_position == 0:
            rodada = row.text
            data = [tournament, season, rodada]
            th_position = 1
        elif row.name == 'td':
            if td_position == 0:
                data.append(row.text.split(' - ')[0])
                data.append(row.text.split(' - ')[1])
                for a in row.find_all('a', href=True):
                    link = a['href']
                    #df_goals = scrapGoals(len(df),[row.text.split(' - ')[0],row.text.split(' - ')[1]],link, df_goals)
            elif 2 <= td_position <= 4:
                try:
                    data.append(row.select('span')[2].attrs['data-odd'])
                except:
                    try: 
                        data.append(row.attrs['data-odd'])
                    except:
                        data.append(row.text)                    
            else:
                data.append(row.text)
            td_position+=1       
            if td_position == 6:
                data.append(link)
                data.append('')
                df.loc[len(df)] = data
                data = [tournament, season, rodada]
                th_position = 0
                td_position = 0
    #return df, df_goals
    return df

def scrapGoals(match_id, team_id, page_link, df_goals):
    page_content = get_page(page_link)
    for ul in page_content.find_all('ul', attrs = {'class' : 'list-details list-details--shooters'}):
        i = 0
        for li in ul.findAll('li'):
            if len(li) > 1:
                for tr in li.findAll('tr'):
                    #home team
                    if i == 0:
                        df_goals.loc[len(df_goals)] = [match_id, team_id[0], tr.text.split('.')[0],tr.text.split('.')[1]]
                    #away team
                    else:
                        df_goals.loc[len(df_goals)] = [match_id, team_id[1], tr.text.split('.')[0],tr.text.split('.')[1]]
            i = 1

    return df_goals

In [25]:
df_tournaments_links = pd.read_excel('data/Tournament Links.xlsx')
log_columns = list(df_tournaments_links.columns)
log_columns.append('Error')
log = pd.DataFrame(columns= log_columns)

In [27]:
#seleciona torneios a serem coletados
df_tournaments_links = df_tournaments_links[(df_tournaments_links.Season.isin(['2016','2017'])) & (df_tournaments_links.Country == 'Brazil')]

In [28]:
%%time
current_country = df_tournaments_links.loc[df_tournaments_links.index.min()].Country
df_country_competitions = pd.DataFrame(columns=['Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
for row in df_tournaments_links.values:
    row = list(row)
    page_link = row[3]
    try:
        page_content = get_page(page_link+'results')
    except:
        row.append('Timeout')
        log.loc[len(log)] = row
        break
    div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
    try:
        if len(div) > 0:
            row.append('scrapGroupPhase')
            df = scrapGroupPhase(div, page_link, row[1], row[2])
            #df, df_goals = scrapGroupPhase(div, page_link)
        else:
            row.append('scrapChampionship')
            #df, df_goals = scrapChampionship(page_content)
            df = scrapChampionship(page_content, row[1], row[2])        
        #if len(df_goals) > 0:
            #df_goals.to_excel('data/'+row[1]+' - '+row[2].split('/')[0]+' - Goals.xlsx', index=False)
        if row[0] == current_country:
            df_country_competitions = df_country_competitions.append(df)
        else:
            df_country_competitions.to_excel('data/'+current_country+'.xlsx', index=False)
            current_country = row[0]
    except:
        log.loc[len(log)] = row    
df_country_competitions.to_excel('data/'+current_country+'.xlsx', index=False)
log.to_csv('data/log.csv', index=False)

['Serie C', '2017', 'Group A - 18. Round', 'ASA', 'Confianca', '1:2', '4.76', '3.51', '1.62', '10.09.2017', '/soccer/brazil/serie-c-2017/asa-confianca/ENIfSJHA/', 'Main']
['Serie C', '2017', 'Main', 'Cuiaba Esporte', 'CSA', '1:1', '1.64', '3.34', '5.05', '10.09.2017', '/soccer/brazil/serie-c-2017/cuiaba-esporte-csa/C8ybRwXG/', 'Main']
['Serie C', '2017', 'Main', 'Fortaleza', 'Moto Club', '1:0', '1.58', '3.37', '5.65', '10.09.2017', '/soccer/brazil/serie-c-2017/fortaleza-moto-club/pMZ5PH1T/', 'Main']
['Serie C', '2017', 'Main', 'Salgueiro', 'Remo', '2:1', '2.07', '3.14', '3.21', '10.09.2017', '/soccer/brazil/serie-c-2017/salgueiro-remo/6Vz2QcnN/', 'Main']
['Serie C', '2017', 'Main', 'Sampaio Correa', 'Botafogo PB', '2:3', '1.98', '3.27', '3.34', '10.09.2017', '/soccer/brazil/serie-c-2017/sampaio-correa-botafogo-pb/WbTkTa24/', 'Main']
['Serie C', '2017', 'Group B - 18. Round', 'Botafogo SP', 'Ypiranga FC', '5:3', '1.74', '3.33', '4.43', '09.09.2017', '/soccer/brazil/serie-c-2017/botafogo

['Serie C', '2017', 'Final - 2nd leg', 'CSA', 'Fortaleza', '0:0', '2.08', '3.19', '3.19', '21.10.2017', '/soccer/brazil/serie-c-2017/csa-fortaleza/tWRIcnvh/', 'Play Offs']
['Serie C', '2017', 'Final - 1st leg', 'Fortaleza', 'CSA', '1:2', '2.12', '2.99', '3.24', '15.10.2017', '/soccer/brazil/serie-c-2017/fortaleza-csa/SSNEbSOo/', 'Play Offs']
['Serie C', '2017', 'Semi-finals - 2nd leg', 'CSA', 'Sao Bento', '1:1 PEN.', '1.78', '3.10', '4.52', '08.10.2017', '/soccer/brazil/serie-c-2017/csa-sao-bento/6kmI0a7M/', 'Play Offs']
['Serie C', '2017', 'Play Offs', 'Sampaio Correa', 'Fortaleza', '2:2', '1.97', '3.16', '3.52', '07.10.2017', '/soccer/brazil/serie-c-2017/sampaio-correa-fortaleza/SblE1uhG/', 'Play Offs']
['Serie C', '2017', 'Semi-finals - 1st leg', 'Fortaleza', 'Sampaio Correa', '1:0', '1.73', '3.31', '4.54', '03.10.2017', '/soccer/brazil/serie-c-2017/fortaleza-sampaio-correa/WWf531N3/', 'Play Offs']
['Serie C', '2017', 'Play Offs', 'Sao Bento', 'CSA', '0:1', '1.89', '3.02', '4.15', '

['Serie D', '2017', 'Main', 'Tocantins', 'Santos AP', '0:1', '\xa0', '\xa0', '\xa0', '18.06.2017', '/soccer/brazil/serie-d-2017/tocantins-santos-ap/jerLFjnL/', 'Main']
['Serie D', '2017', 'Group 3 - 5. Round', 'Rio Branco', 'Sao Francisco', '0:0', '\xa0', '\xa0', '\xa0', '18.06.2017', '/soccer/brazil/serie-d-2017/rio-branco-sao-francisco/Cr8t4gm9/', 'Main']
['Serie D', '2017', 'Group 2 - 5. Round', 'SAO Raimundo PA', 'Fast Clube', '1:1', '\xa0', '\xa0', '\xa0', '18.06.2017', '/soccer/brazil/serie-d-2017/sao-raimundo-pa-fast-clube/GbVcDFAG/', 'Main']
['Serie D', '2017', 'Group 1 - 5. Round', 'Princesa Solimoes', 'Acreano', '1:0', '\xa0', '\xa0', '\xa0', '18.06.2017', '/soccer/brazil/serie-d-2017/princesa-solimoes-acreano/hdI72bdb/', 'Main']
['Serie D', '2017', 'Group 16 - 5. Round', 'Novo Hamburgo', 'Foz do Iguacu', '1:1', '1.56', '3.54', '5.14', '18.06.2017', '/soccer/brazil/serie-d-2017/novo-hamburgo-foz-do-iguacu/vuaTPMhk/', 'Main']
['Serie D', '2017', 'Group 15 - 5. Round', 'Operari

['Serie D', '2017', 'Group 7 - 3. Round', 'Central SC', 'Juazeirense', '1:1', '2.30', '3.35', '2.62', '04.06.2017', '/soccer/brazil/serie-d-2017/central-sc-juazeirense/vDvlzIT4/', 'Main']
['Serie D', '2017', 'Main', 'Sousa', 'Coruripe', '1:1', '\xa0', '\xa0', '\xa0', '04.06.2017', '/soccer/brazil/serie-d-2017/sousa-coruripe/I3whZyqB/', 'Main']
['Serie D', '2017', 'Group 6 - 3. Round', 'America PE', 'Globo', '1:0', '2.41', '3.12', '2.63', '04.06.2017', '/soccer/brazil/serie-d-2017/america-pe-globo/Yg05P0Fn/', 'Main']
['Serie D', '2017', 'Main', 'Guarani de Juazeiro', 'Parnahyba', '1:0', '1.90', '3.36', '3.44', '04.06.2017', '/soccer/brazil/serie-d-2017/guarani-de-juazeiro-parnahyba/Cpa9OKUh/', 'Main']
['Serie D', '2017', 'Group 5 - 3. Round', 'Maranhao', 'Guarany', '1:0', '2.25', '3.26', '2.75', '04.06.2017', '/soccer/brazil/serie-d-2017/maranhao-guarany/ANsdD4Lt/', 'Main']
['Serie D', '2017', 'Main', 'Potiguar', 'River-PI', '0:2', '\xa0', '\xa0', '\xa0', '04.06.2017', '/soccer/brazil/s

['Serie D', '2017', 'Group 4 - 2. Round', 'AE Altos', 'Cordino EC', '4:0', '1.44', '4.17', '5.66', '27.05.2017', '/soccer/brazil/serie-d-2017/ae-altos-cordino-ec/4tmLdBuq/', 'Main']
['Serie D', '2017', 'Group 15 - 2. Round', 'Brusque', 'Piracicaba', '3:1', '1.81', '3.41', '3.72', '27.05.2017', '/soccer/brazil/serie-d-2017/brusque-piracicaba/bereZj11/', 'Main']
['Serie D', '2017', 'Group 17 - 2. Round', 'EC Sao Jose', 'Metropolitano', '4:0', '1.73', '3.46', '4.07', '27.05.2017', '/soccer/brazil/serie-d-2017/ec-sao-jose-metropolitano/d07S4rin/', 'Main']
['Serie D', '2017', 'Group 16 - 2. Round', 'Sao Bernardo', 'Inter de Lages', '1:0', '1.65', '3.46', '4.66', '27.05.2017', '/soccer/brazil/serie-d-2017/sao-bernardo-inter-de-lages/0KlP9XxE/', 'Main']
['Serie D', '2017', 'Final - 2nd leg', 'Operario', 'Globo', '0:1', '1.71', '3.42', '4.14', '11.09.2017', '/soccer/brazil/serie-d-2017/operario-globo/EHRKja4i/', 'Play Offs']
['Serie D', '2017', 'Final - 1st leg', 'Globo', 'Operario', '0:5', '2

...

['Campeonato Potiguar', '2016', 'First stage', 'Potiguar', 'Baraunas', '1:0', '\xa0', '\xa0', '\xa0', '31.01.2016', '/soccer/brazil/campeonato-potiguar-2016/potiguar-baraunas/YHlHPVf8/', 'First stage']
['Campeonato Potiguar', '2016', '2. Round', 'Baraunas', 'Alecrim', '2:0', '\xa0', '\xa0', '\xa0', '28.01.2016', '/soccer/brazil/campeonato-potiguar-2016/baraunas-alecrim/hMBU1S21/', 'First stage']
['Campeonato Potiguar', '2016', 'First stage', 'Palmeira de Una', 'Potiguar', '0:3', '\xa0', '\xa0', '\xa0', '28.01.2016', '/soccer/brazil/campeonato-potiguar-2016/palmeira-de-una-potiguar/On7Q28me/', 'First stage']
['Campeonato Potiguar', '2016', 'First stage', 'America RN', 'Assu RN', '4:0', '\xa0', '\xa0', '\xa0', '27.01.2016', '/soccer/brazil/campeonato-potiguar-2016/america-rn-assu-rn/pd8M3lYl/', 'First stage']
['Campeonato Potiguar', '2016', 'First stage', 'Globo', 'ABC', '3:1', '\xa0', '\xa0', '\xa0', '26.01.2016', '/soccer/brazil/campeonato-potiguar-2016/globo-abc/2XIH4UIr/', 'First sta

['Campeonato Sergipano', '2016', 'First stage', 'Lagarto', 'Amadense', '1:1', '\xa0', '\xa0', '\xa0', '11.02.2016', '/soccer/brazil/campeonato-sergipano-2016/lagarto-amadense/tnAr9FZm/', 'First stage']
['Campeonato Sergipano', '2016', 'First stage', 'Sergipe', 'Estanciano EC', '1:2', '\xa0', '\xa0', '\xa0', '11.02.2016', '/soccer/brazil/campeonato-sergipano-2016/sergipe-estanciano-ec/OAFwAeKs/', 'First stage']
['Campeonato Sergipano', '2016', 'First stage', 'Socorrense', 'Dorense', '1:1', '\xa0', '\xa0', '\xa0', '11.02.2016', '/soccer/brazil/campeonato-sergipano-2016/socorrense-dorense/Ea8j7g40/', 'First stage']
['Campeonato Sergipano', '2016', '5. Round', 'Amadense', 'Confianca', '1:3', '\xa0', '\xa0', '\xa0', '06.02.2016', '/soccer/brazil/campeonato-sergipano-2016/amadense-confianca/0hCF5tTe/', 'First stage']
['Campeonato Sergipano', '2016', 'First stage', 'Dorense', 'Itabaiana', '2:1', '\xa0', '\xa0', '\xa0', '06.02.2016', '/soccer/brazil/campeonato-sergipano-2016/dorense-itabaiana/

['Campeonato Sul-Matogrossense', '2016', 'Group B - 10. Round', 'Aquidauanense', 'Aguia Negra', '0:1', '\xa0', '\xa0', '\xa0', '03.04.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/aquidauanense-aguia-negra/xz1ycnMB/', 'Main']
['Campeonato Sul-Matogrossense', '2016', 'Main', 'Ivinhema', '7 de Setembro', '5:1', '\xa0', '\xa0', '\xa0', '03.04.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/ivinhema-7-de-setembro/fgWKP6uT/', 'Main']
['Campeonato Sul-Matogrossense', '2016', 'Main', 'Naviraiense', 'Corumbaense', '0:2', '\xa0', '\xa0', '\xa0', '03.04.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/naviraiense-corumbaense/rJSGQnQM/', 'Main']
['Campeonato Sul-Matogrossense', '2016', 'Group A - 10. Round', 'Costa Rica', 'Comercial MS', '0:1', '\xa0', '\xa0', '\xa0', '03.04.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/costa-rica-comercial-ms/jaKAOKr1/', 'Main']
['Campeonato Sul-Matogrossense', '2016', 'Main', 'Misto', 'Novoperario', '0:0', '\xa0', '\xa0', 

['Campeonato Sul-Matogrossense', '2016', 'Final - 2nd leg', 'Comercial MS', '7 de Setembro', '0:2', '\xa0', '\xa0', '\xa0', '08.05.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/comercial-ms-7-de-setembro/EZ4mf8Bi/', 'Play Offs']
['Campeonato Sul-Matogrossense', '2016', 'Final - 1st leg', '7 de Setembro', 'Comercial MS', '2:1', '\xa0', '\xa0', '\xa0', '01.05.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/7-de-setembro-comercial-ms/zq6qeldo/', 'Play Offs']
['Campeonato Sul-Matogrossense', '2016', 'Semi-finals - 2nd leg', '7 de Setembro', 'Operario MS', '0:0', '\xa0', '\xa0', '\xa0', '24.04.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/7-de-setembro-operario-ms/G4CKMkEj/', 'Play Offs']
['Campeonato Sul-Matogrossense', '2016', 'Play Offs', 'Comercial MS', 'Corumbaense', '0:0', '\xa0', '\xa0', '\xa0', '24.04.2016', '/soccer/brazil/campeonato-sul-matogrossense-2016/comercial-ms-corumbaense/4U8GNVbp/', 'Play Offs']
['Campeonato Sul-Matogrossense', '2016', 'Semi

['Copa do Nordeste', '2016', 'Group B - 5. Round', 'Estanciano EC', 'America RN', '0:0', '4.92', '3.80', '1.57', '08.03.2016', '/soccer/brazil/copa-do-nordeste-2016/estanciano-ec-america-rn/xANJMZW6/', 'Main']
['Copa do Nordeste', '2016', 'Group E - 4. Round', 'Flamengo PI', 'Vitoria da Conquista', '0:0', '3.28', '3.06', '2.11', '04.03.2016', '/soccer/brazil/copa-do-nordeste-2016/flamengo-pi-vitoria-da-conquista/ObBY446c/', 'Main']
['Copa do Nordeste', '2016', 'Group D - 4. Round', 'Botafogo PB', 'Fortaleza', '2:1', '2.94', '3.13', '2.22', '03.03.2016', '/soccer/brazil/copa-do-nordeste-2016/botafogo-pb-fortaleza/MZ44UNUp/', 'Main']
['Copa do Nordeste', '2016', 'Main', 'Sport Recife', 'River-PI', '2:2', '1.24', '4.71', '11.60', '03.03.2016', '/soccer/brazil/copa-do-nordeste-2016/sport-recife-river-pi/Akbng77o/', 'Main']
['Copa do Nordeste', '2016', 'Group C - 4. Round', 'Confianca', 'Bahia', '0:3', '3.39', '3.17', '2.00', '03.03.2016', '/soccer/brazil/copa-do-nordeste-2016/confianca-bah

In [24]:
%%time
#célula de testes
'''page_link = '/soccer/brazil/campeonato-mineiro-2017/results/'
page_content = get_page(page_link)
print('Page')
div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
if len(div) > 0:
    print('Group')
    #df, df_goals = scrapGroupPhase(div, page_link)
    df = scrapGroupPhase(div, page_link, 'Primeira Liga', '2017')
else:
    print('Champ')
    #df, df_goals = scrapChampionship(page_content)
    df = scrapChampionship(page_content, 'Primeira Liga', '2017')'''

Page
Group
['Primeira Liga', '2017', '11. Round', 'America MG', 'Villa Nova MG', '1:1', '1.44', '3.96', '6.25', '09.04.2017', '/soccer/brazil/campeonato-mineiro-2017/america-mg-villa-nova-mg/d0Ljq1e6/', 'Main']
['Primeira Liga', '2017', 'Main', 'America TO', 'Tricordiano', '4:1', '2.70', '3.01', '2.46', '09.04.2017', '/soccer/brazil/campeonato-mineiro-2017/america-to-tricordiano/fyPZnqds/', 'Main']
['Primeira Liga', '2017', 'Main', 'Caldense', 'Atletico-MG', '2:1', '2.30', '3.24', '2.74', '09.04.2017', '/soccer/brazil/campeonato-mineiro-2017/caldense-atletico-mg/0YRsoNQg/', 'Main']
['Primeira Liga', '2017', 'Main', 'Cruzeiro', 'EC Democrata', '2:0', '1.27', '4.86', '8.93', '09.04.2017', '/soccer/brazil/campeonato-mineiro-2017/cruzeiro-ec-democrata/6BSwn3Bm/', 'Main']
['Primeira Liga', '2017', 'Main', 'Tombense', 'URT', '1:2', '2.21', '3.08', '3.01', '09.04.2017', '/soccer/brazil/campeonato-mineiro-2017/tombense-urt/ziKfrLAC/', 'Main']
['Primeira Liga', '2017', 'Main', 'Uberlandia', 'Tu

['Primeira Liga', '2017', 'Main', 'Uberlandia', 'EC Democrata', '2:0', '1.50', '3.59', '6.23', '05.02.2017', '/soccer/brazil/campeonato-mineiro-2017/uberlandia-ec-democrata/0xIg2L0h/', 'Main']
['Primeira Liga', '2017', 'Main', 'Caldense', 'Villa Nova MG', '1:0', '1.90', '3.23', '3.66', '04.02.2017', '/soccer/brazil/campeonato-mineiro-2017/caldense-villa-nova-mg/W2Dk31on/', 'Main']
['Primeira Liga', '2017', 'Main', 'Tombense', 'Atletico-MG', '0:3', '5.23', '3.73', '1.54', '04.02.2017', '/soccer/brazil/campeonato-mineiro-2017/tombense-atletico-mg/GEqypvoO/', 'Main']
['Primeira Liga', '2017', '1. Round', 'EC Democrata', 'America MG', '0:2', '2.91', '3.24', '2.19', '29.01.2017', '/soccer/brazil/campeonato-mineiro-2017/ec-democrata-america-mg/v9lCkrHn/', 'Main']
['Primeira Liga', '2017', 'Main', 'Tupi', 'Tombense', '0:1', '2.32', '3.06', '2.83', '29.01.2017', '/soccer/brazil/campeonato-mineiro-2017/tupi-tombense/hChGl2Wh/', 'Main']
['Primeira Liga', '2017', 'Main', 'URT', 'Caldense', '1:0',

In [32]:
df_country_competitions.to_csv('data/'+current_country+'.csv', index=False)